<a href="https://colab.research.google.com/github/gerbeldo/rtccnn/blob/main/notebooks/colab-imports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

## data

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")
GDRIVE_PATH = "/content/gdrive/MyDrive/austral/tesis"
GDRIVE_DATA_PATH = f"{GDRIVE_PATH}/data"
GDRIVE_TRAINED_MODELS_PATH = f"{GDRIVE_PATH}/trained_models"

!cp $GDRIVE_DATA_PATH/processed_data.tar.gz processed_data.tar.gz
!tar -xf ./processed_data.tar.gz

!mkdir "./trained_models"

Mounted at /content/gdrive
tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'


## Modules

In [3]:
# add path to modules to be able to import stuff
import sys
from pathlib import Path

HOME = Path.cwd()
REPO_PATH = HOME / "rtccnn"
CONFIG_PATH = REPO_PATH / "config" / "colab.yaml"

!git clone https://github.com/gerbeldo/rtccnn.git $REPO_PATH

# Get the absolute path to the 'utils' directory
utils_path = REPO_PATH / "utils"
models_path = REPO_PATH / "models"

# Add this path to sys.path
if utils_path not in sys.path:
    sys.path.append(str(utils_path.resolve()))

if models_path not in sys.path:
    sys.path.append(str(models_path.resolve()))

Cloning into '/content/rtccnn'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 99 (delta 52), reused 75 (delta 31), pack-reused 0
Receiving objects: 100% (99/99), 23.80 KiB | 11.90 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# Import modules

In [4]:
from load_config import load_config
from dataset import CellDivisionDataset
from train import train_model
from simple_3d_cnn import Simple3DCNN

In [ ]:
from getpass import getpass
import neptune
from neptune_pytorch import NeptuneLogger

my_api_token = getpass("Enter your Neptune API token: ")
my_project = "gerbeldo/rtccnn"
run = neptune.init_run(project=my_project, api_token=my_api_token)

# Train model

In [ ]:
# init config dataset and model
config = load_config(str(CONFIG_PATH))
config["train"]["epochs"] = 1

dataset = CellDivisionDataset(
    config["data"]["annotations_path"],
    config["data"]["img_path"],
    transform=None,
    device=config["env"]["device"],
)

model = Simple3DCNN()

In [ ]:
# init neptune stuff

npt_logger = NeptuneLogger(
    run=run,
    model=model,
    log_model_diagram=True,
    log_gradients=True,
    log_parameters=True,
    log_freq=10,
)
run["config"] = config

In [6]:
model = train_model(config, dataset, model, run, npt_logger)

Model loaded on cuda
Using GPU: Tesla T4
Epoch 1, CUDA Memory allocated: 590.85 MB
Batch 1: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 2: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 3: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 4: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 5: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 6: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 7: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 8: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 9: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 10: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
[1,    10] loss: 0.845
Batch 11: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 12: Input tensor shape torch.Size([32, 1, 20, 75, 75]) on cuda:0
Batch 13: Input tensor shape torch.Size([32, 1, 20,

# Save model

In [22]:
from datetime import datetime
import torch

model_id = datetime.now().strftime("%Y%m%d-%H%M%S") + "_model.pt"
model_path = Path("trained_models") / model_id

torch.save(model, model_path)

# upload to drive
!cp $model_path /content/gdrive/MyDrive/austral/tesis/trained_models/{model_id}

In [ ]:
npt_logger.log_model("model")
run.stop()